In [1]:
import numpy as np
import pandas as pd
import anndata
import matplotlib.pyplot as plt
import spatialdata as sd
import scanpy as sc

In [ ]:
adata = sc.read_h5ad('/storage/lingyuan2/STATES_data/mousebrain_withannotation_20250614.h5ad')

In [ ]:
adata

In [ ]:
df = adata.uns['remain_reads_info_new']
# Get list of detected genes from adata.var
detected_genes = adata.var[adata.var['detected']].index.tolist()

# Filter df to only include detected genes
df = df[df['gene'].isin(detected_genes)]
df

In [ ]:
# Group by gene and cell_barcode type (-1 vs non -1)
processes = df[df['cell_barcode'] == -1].groupby(['gene', 'type']).size().unstack(fill_value=0)
somata = df[df['cell_barcode'] != -1].groupby(['gene', 'type']).size().unstack(fill_value=0)

# Calculate rbRNA/(rbRNA+ntRNA) TE for each group
processes['TE'] = processes['rbRNA'] / (processes['rbRNA'] + processes['ntRNA'])
somata['TE'] = somata['rbRNA'] / (somata['rbRNA'] + somata['ntRNA'])

# Combine results into a single dataframe
result = pd.DataFrame({
    'processes_ntRNA': processes['ntRNA'],
    'processes_rbRNA': processes['rbRNA'], 
    'processes_TE': processes['TE'],
    'somata_ntRNA': somata['ntRNA'],
    'somata_rbRNA': somata['rbRNA'],
    'somata_TE': somata['TE']
})

result

In [ ]:
result['fc'] = result['processes_TE'] / result['somata_TE']
result['log2fc'] = np.log2(result['fc'])
result

In [ ]:
result.to_csv('soma_process_eachgene_all_0503.csv')

In [2]:
import pandas as pd

df = pd.read_csv('/storage/lingyuan2/STATES_data/soma_process_eachgene_all_0503.csv', index_col=0)

df_fdr = pd.read_csv('/storage/lingyuan2/STATES_data/gene_TE_comparison_results_with_FDR_all0503.csv')

df_fdr = df_fdr.set_index('gene')

df['adj_pvalue'] = df_fdr.loc[df.index, 'adj_pvalue']
df['significant'] = df_fdr.loc[df.index, 'significant']

df.to_csv('/storage/lingyuan2/STATES_data/eachgenefinal_all0503.csv')
